In [28]:
import re
import pandas as pd
import warnings
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

warnings.filterwarnings("ignore")

### 네이버 영화 평점 데이터 탐색
* id : 각 데이터의 id
* document : 영화평의 텍스트
* label : 영화평(document)의 Sentiment 결과 값(Target label)
    * 1 : 긍정적 평가
    * 0 : 부정적 평가

In [8]:
train_df = pd.read_csv('./nsmc-master/ratings_train.txt', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


* train_df에서 documnet가 Null인 데이터가 존재 이에대한 처리 필요

In [13]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


* 0(부정적 평가)과 1(긍정적 평가)의 비율이 어느 한쪽으로 치우치지 않고 균등한 분포를 가짐

In [9]:
train_df['label'].value_counts( )

0    75173
1    74827
Name: label, dtype: int64

### 데이터 클렌징
* train_df의 경우 리뷰 텍스트를 가지는 'document'컬럼에 Null이 일부 존재하므로 이 값을 공백으로 변환
* 문자가 아닌 숫자의 경우 단어적인 의미로 부족하므로 정규식을 활용하여 공백으로 변환

In [20]:
train_df = train_df.fillna(' ')
# 정규 표현식을 이용하여 숫자를 공백으로 변경(정규 표현식으로 \d 는 숫자를 의미함.) 
train_df['document'] = train_df['document'].apply( lambda x : re.sub(r"\d+", " ", x) )

# 테스트 데이터 셋을 로딩하고 동일하게 Null 및 숫자를 공백으로 변환
test_df = pd.read_csv('./nsmc-master/ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply( lambda x : re.sub(r"\d+", " ", x) )

# 개정판 소스 코드 변경(2019.12.24)
train_df.drop('id', axis=1, inplace=True) 
test_df.drop('id', axis=1, inplace=True)


#### 한글 형태소 분석을 통한 형태소 단어로 토큰화
* SNS 분석에 적합합 Twitter 클래스를 한글 형태소 엔진으로 사용
* Twitter.morphs() : 입력 인자로 들어온 문장을 형태소 단어 형태로 토큰화하여 list객체로 반환

#### TF-IDF 형태로 피처 벡터화
* tokenizer : 토큰화를 별도의 커스텀 함수로 사용할 경우 사용
    * 별도로 정의한 tw_tokenizer 사용
* ngram_range : BOW 모델의 단어 순서를 어느 정도 보강하기 위한 n_grma범위
    * (1,2) : 토큰화된 단어를 최소 1개씩 최대 2개씩 순서대로 묶어 피처로 추출
* min_df : 전체 문서에 걸쳐서 너무 낮은 빈도수를 가지는 단어 피처를 제외하기 위한 파라미터
    * 3 : 전체 문서에 걸쳐 하위 3개 이하의 빈도수를 가지는 단어는 피처에서 제외
* max_df : 전체 문서에 걸쳐서 너무 높은 빈도수를 가지는 단어 피처를 제외하기 위한 파라미터
    * 0.9 : 부동 소수점값을 가지면 전체 문서에 걸쳐 상위 10% 는 피처에서 제외

In [22]:
twitter = Twitter()
def tw_tokenizer(text):
    # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = twitter.morphs(text)
    return tokens_ko

# Twitter 객체의 morphs( ) 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2) 
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])
# 학습 데이터를 적용한 TfidfVectorizer를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환함. 
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

### 머신러닝 모델 학습/예측/평가
#### Logistic Regression 하이퍼 파라미터 튜닝
* GredSearch로 LogisticRegression의 C 파라미터의 최적의 파라미터를 찾음
    * C : Regularization 강도를 조절하는 alpha값의 역수, 작을 수록 규제 강도가 큼

In [24]:
# Logistic Regression 을 이용하여 감성 분석 Classification 수행. 
lg_clf = LogisticRegression(random_state=0)

# Parameter C 최적화를 위해 GridSearchCV 를 이용. 
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
grid_cv = GridSearchCV(lg_clf , param_grid=params , cv=3 ,scoring='accuracy', verbose=1 )
grid_cv.fit(tfidf_matrix_train , train_df['label'])
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))
print('GridSearchCV 최적 파라미터:', grid_cv.best_params_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 3.5} 0.8593
GridSearchCV 최적 파라미터: {'C': 3.5}


In [26]:
# classifier 는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)
print('Logistic Regression 정확도: ',accuracy_score(test_df['label'],preds))

Logistic Regression 정확도:  0.86184


#### TfidfVectorizer 하이퍼 파라미터 튜닝, 최종 모델의 Test set 감정 예측 및 정확도 평가
* scikit-learn Pipline 클래스를 사용하여 피처 벡터화와 ML 알고리즘 학습/예측을 한번에 수행 TfidfVectorizer의 최적 파라미터를 찾음
* Logistic Regression 의 C 파라미터를 최적 파라미터인 3.5로 고정
* GredSearch로 TfidfVectorizer의 ngram_range, max_df 와 min_df 최적의 파라미터를 찾음
    * ngram_range : BOW 모델의 단서 순서 보강을 위한 n_gram 범위 지정
    * max_df : 전체 문서에 걸처서 너무 높은 빈도수를 가지는 단어 피처를 제외하기 위한 파라미터
    * min_df : 전체 문서에 걸쳐서 너무 낮은 빈도수를 가지는 단어 피처를 제외하기 위한 파라미터

In [ ]:
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(tokenizer=tw_tokenizer)),
    ('lr_clf', LogisticRegression(C = grid_cv.best_params_['C'], random_state=0))
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_)2개를 연달아 붙여 GridSearchCV에 사용될 
# 파라미터/하이퍼 파라미터 이름과 값을 설정. . 
params = { 'tfidf_vect__ngram_range' : [(1,2), (1,3)],
           'tfidf_vect__max_df' : [0.85, 0.9, 0.95],
           'tfidf_vect__min_df' : [3, 5, 10]
}

# Pipeline 객체에 대한 GridSearchCV 수행
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3 , scoring='accuracy', verbose=1)
grid_cv_pipe.fit(train_df['document'] , train_df['label'])
print('Pipeline을 통한 TfidfVectorizer의 최적 파라미터', grid_cv_pipe.best_params_ )
print('Pipeline을 통한 최대 예측 정확도', grid_cv_pipe.best_score_ )

In [36]:
pred = grid_cv_pipe.predict(test_df['document'])
print('최종 Logistic Regression 의 Test dataset 예측 정확도는 {0:.3f}'.format(accuracy_score(test_df['label'] ,pred)))

최종 Logistic Regression 의 Test dataset 예측 정확도는 0.864
